**Course:** SMM  
**Assignment:** 3

## Team Members
- **Muhammad Rehan** - 02-205252-012  
- **Arzoo Ghani Khan** - 02-205252-003

## Question 1

### Dataset
Raw dataset from: [Kaggle — datascientist97/university-students-marks-sheet](https://www.kaggle.com/datascientist97/university-students-marks-sheet)

### Github
Repo: [MuhammadRehanRasool/masters-smm-a3](https://github.com/MuhammadRehanRasool/masters-smm-a3)

# 1. Clean Data and Prepare Final Dataset

In [193]:
# Load Dataset
import pandas as pd

xls = pd.ExcelFile('student_scores_raw.xls')
print("Sheets:", xls.sheet_names)

# Read second and third sheets by index (1 and 2)
sessional = pd.read_excel(xls, sheet_name=1)
final = pd.read_excel(xls, sheet_name=2)

print("Sheet 2 preview:")
print(sessional.head())
print("\nSheet 3 preview:")
print(final.head())

Sheets: ['Table', 'Sessional', 'Final']
Sheet 2 preview:
  Roll No           Name Quiz 15  Unnamed: 3 Unnamed: 4 Assignments 15   \
0     NaN            NaN  Quiz 01    Quiz 02   AVG (15)       Assig. 01   
1       1  Ashir Mehfooz       14         14         14              13   
2       2    Atif Raftad        4         10          7               4   
3       3     Saiqa Aziz       15         11         13              14   
4       8   Ozair Minhas        6          5        5.5               4   

  Unnamed: 6 Unnamed: 7 Mid Term    Q+A+Mid  
0   Assig.02   AVG (15)       45   Total 75  
1         13         13       41         68  
2          5        4.5       30       41.5  
3         13       13.5       34       60.5  
4          6          5       12       22.5  

Sheet 3 preview:
  Roll No. Name of Student Total Marks (150)     Unnamed: 3 Unnamed: 4  \
0      NaN             NaN    Sessional (75)  Terminal (75)  Total 150   
1        1   Ashir Mehfooz                68      

In [194]:
# Drop first row of both dataframes
sessional = sessional.drop(index=0).reset_index(drop=True)
final = final.drop(index=0).reset_index(drop=True) 

In [195]:
# Create new combined dataframe with specified columns
combined = pd.DataFrame()

# 1. Attendance (null initially)
combined['attendance'] = None

# 2. Quizzes (total 15marks)
combined['quizzes'] = pd.to_numeric(sessional.iloc[:, 4], errors='coerce')

# 3. Assignment (total 15marks)
combined['assignment'] = pd.to_numeric(sessional.iloc[:, 7], errors='coerce')

# 4. Final (total 75marks)
combined['final'] = pd.to_numeric(final.iloc[:, 3], errors='coerce')

# 5. GPA (0-4 scale)
combined['gpa'] = pd.to_numeric(final.iloc[:, -1], errors='coerce')

# 6. Midterm (total 45marks)
combined['midterm'] = pd.to_numeric(sessional.iloc[:, 8], errors='coerce')

print("Combined dataframe:")
print(combined.head(10))
print("\nDataframe info:")
print(combined.info())
print("\nDataframe statistics:")
print(combined.describe())

Combined dataframe:
  attendance  quizzes  assignment  final  gpa  midterm
0        NaN     14.0        13.0   55.0  4.0     41.0
1        NaN      7.0         4.5   45.0  2.5     30.0
2        NaN     13.0        13.5   58.0  3.9     34.0
3        NaN      5.5         5.0   44.0  0.0     12.0
4        NaN      8.0         4.5   35.0  2.2     34.0
5        NaN     13.5         3.5   45.0  2.8     33.0
6        NaN     12.0        12.5   66.0  NaN      NaN
7        NaN      7.5         4.0   43.0  2.6     34.0
8        NaN     10.0         5.0   23.0  0.0     25.0
9        NaN     14.0         6.0   22.0  0.0     23.0

Dataframe info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 224 entries, 0 to 223
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   attendance  0 non-null      object 
 1   quizzes     216 non-null    float64
 2   assignment  216 non-null    float64
 3   final       213 non-null    float64
 4   gpa

In [196]:
# Check for missing values
print("\nMissing values in each column:")
print(combined.isnull().sum())


Missing values in each column:
attendance    224
quizzes         8
assignment      8
final          11
gpa            19
midterm        19
dtype: int64


In [197]:
# Cap the maximum values to their respective totals
combined.loc[combined['quizzes'] > 15, 'quizzes'] = 15
combined.loc[combined['assignment'] > 15, 'assignment'] = 15
combined.loc[combined['final'] > 75, 'final'] = 75
combined.loc[combined['midterm'] > 45, 'midterm'] = 45

In [198]:
# Except attendance and GPA, fill missing values with mean for numerical columns
combined['quizzes'].fillna(combined['quizzes'].mean(), inplace=True)
combined['assignment'].fillna(combined['assignment'].mean(), inplace=True)
combined['final'].fillna(combined['final'].mean(), inplace=True)
combined['midterm'].fillna(combined['midterm'].mean(), inplace=True)
print("\nAfter filling missing values:")
print(combined.isnull().sum())


After filling missing values:
attendance    224
quizzes         0
assignment      0
final           0
gpa            19
midterm         0
dtype: int64


In [199]:
# Show midterm statistics
print("\nMidterm statistics:")
print(combined['midterm'].describe())


Midterm statistics:
count    224.000000
mean      29.556098
std        9.613752
min       11.000000
25%       23.000000
50%       30.000000
75%       35.000000
max       45.000000
Name: midterm, dtype: float64


In [200]:
# Re-calculate GPA based on total marks (150) to 4 scale

combined["gpa"] = (
    (combined["quizzes"] + combined["assignment"] + combined["midterm"] + combined["final"]) / 150 * 4
)

In [201]:
# Round off and Range to 0-100 scale
combined['quizzes'] = ((combined['quizzes'] / 15) * 100).round(2)
combined['assignment'] = ((combined['assignment'] / 15) * 100).round(2)
combined['final'] = ((combined['final'] / 75) * 100).round(2)
combined['midterm'] = ((combined['midterm'] / 45) * 100).round(2)

# Round off
combined['gpa'] = combined['gpa'].round(2)

In [202]:
print("Combined dataframe:")
print(combined.head(10))
print("\nDataframe info:")
print(combined.info())
print("\nDataframe statistics:")
print(combined.describe())

Combined dataframe:
  attendance  quizzes  assignment  final   gpa  midterm
0        NaN    93.33       86.67  73.33  3.28    91.11
1        NaN    46.67       30.00  60.00  2.31    66.67
2        NaN    86.67       90.00  77.33  3.16    75.56
3        NaN    36.67       33.33  58.67  1.77    26.67
4        NaN    53.33       30.00  46.67  2.17    75.56
5        NaN    90.00       23.33  60.00  2.53    73.33
6        NaN    80.00       83.33  88.00  3.20    65.68
7        NaN    50.00       26.67  57.33  2.36    75.56
8        NaN    66.67       33.33  30.67  1.68    55.56
9        NaN    93.33       40.00  29.33  1.73    51.11

Dataframe info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 224 entries, 0 to 223
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   attendance  0 non-null      object 
 1   quizzes     224 non-null    float64
 2   assignment  224 non-null    float64
 3   final       224 non-null    float

In [203]:
# Generate realistic attendance based on academic performance
import numpy as np

def generate_attendance(row):
    # Normalize scores to 0-1 scale
    quiz_norm = row['quizzes'] / 100
    assignment_norm = row['assignment'] / 100
    midterm_norm = row['midterm'] / 100
    final_norm = row['final'] / 100
    
    # Weighted average (final exam and GPA have more weight)
    performance_score = (quiz_norm * 0.2 + 
                        assignment_norm * 0.1 + 
                        midterm_norm * 0.3 + 
                        final_norm * 0.4)
    
    # Base attendance: 55-95% range based on performance
    # High performers tend to have 80-95% attendance
    # Low performers tend to have 55-75% attendance
    base_attendance = 55 + (performance_score * 40)
    
    # Add random variation (-5 to +5) to make it more realistic
    variation = np.random.uniform(-5, 5)
    attendance = base_attendance + variation
    
    # Ensure it stays within 0-100 range
    attendance = np.clip(attendance, 0, 100)
    
    return round(attendance, 2)

# Set random seed for reproducibility
np.random.seed(42)

# Generate attendance for each student
combined['attendance'] = combined.apply(generate_attendance, axis=1)

print("Updated dataframe with attendance:")
print(combined.head(10))
print("\nAttendance statistics:")
print(combined['attendance'].describe())
print(f"\nAttendance range: {combined['attendance'].min():.2f}% - {combined['attendance'].max():.2f}%")

Updated dataframe with attendance:
   attendance  quizzes  assignment  final   gpa  midterm
0       87.34    93.33       86.67  73.33  3.28    91.11
1       82.04    46.67       30.00  60.00  2.31    66.67
2       89.29    86.67       90.00  77.33  3.16    75.56
3       72.84    36.67       33.33  58.67  1.77    26.67
4       73.56    53.33       30.00  46.67  2.17    75.56
5       78.09    90.00       23.33  60.00  2.53    73.33
6       82.28    80.00       83.33  88.00  3.20    65.68
7       81.97    50.00       26.67  57.33  2.36    75.56
8       74.25    66.67       33.33  30.67  1.68    55.56
9       76.97    93.33       40.00  29.33  1.73    51.11

Attendance statistics:
count    224.000000
mean      77.100893
std        6.246707
min       58.470000
25%       73.277500
50%       76.580000
75%       81.125000
max       92.510000
Name: attendance, dtype: float64

Attendance range: 58.47% - 92.51%


In [204]:
# Final check for missing values
print("\nMissing values in each column:")
print(combined.isnull().sum())


Missing values in each column:
attendance    0
quizzes       0
assignment    0
final         0
gpa           0
midterm       0
dtype: int64


In [205]:
# Save the cleaned dataset
combined.to_csv('student_scores_clean.csv', index=False)